# Đánh giá thuật toán so sánh ảnh sử dụng Wavelet Transform

### Mục tiêu:
- Sử dụng wavelet để trích xuất đặc trưng từ hình ảnh
- So sánh độ tương đồng giữa các cặp hình ảnh
- Đánh giá hiệu suất thuật toán thông qua:
  - Độ chính xác (Accuracy)
  - Độ nhạy (Sensitivity/Recall)
  - Độ đặc biệt (Specificity)
  - Đường cong ROC

## 1. Import thư viện

In [ ]:
import cv2
import numpy as np
import pywt
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score
import seaborn as sns
from skimage import data
from scipy.spatial.distance import euclidean, cosine
import warnings
warnings.filterwarnings('ignore')

## 2. Hàm trích xuất đặc trưng sử dụng Wavelet Transform

In [ ]:
def extract_wavelet_features(image, wavelet='db1', level=3):
    """
    Trích xuất đặc trưng từ ảnh bằng DWT
    
    Parameters:
        image: Ảnh đầu vào (grayscale)
        wavelet: Loại wavelet (db1, haar, sym, coif)
        level: Số cấp độ phân rã
    
    Returns:
        Vector đặc trưng từ các hệ số wavelet
    """
    # Chuyển sang grayscale nếu cần
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Resize ảnh thành kích thước lũy thừa của 2
    h, w = image.shape
    new_h = 2 ** int(np.ceil(np.log2(h)))
    new_w = 2 ** int(np.ceil(np.log2(w)))
    image_resized = cv2.resize(image, (new_w, new_h))
    
    # Phân rã wavelet 2D
    coeffs = pywt.wavedec2(image_resized, wavelet=wavelet, level=level)
    
    features = []
    
    # Đặc trưng từ hệ số xấp xỉ (cA)
    cA = coeffs[0]
    features.extend([
        np.mean(cA),
        np.std(cA),
        np.median(cA),
        np.max(cA),
        np.min(cA)
    ])
    
    # Đặc trưng từ các hệ số chi tiết (cH, cV, cD)
    for i in range(1, len(coeffs)):
        cH, cV, cD = coeffs[i]
        
        for coeff in [cH, cV, cD]:
            features.extend([
                np.mean(np.abs(coeff)),  # Mean
                np.std(coeff),           # Std
                np.sum(coeff ** 2)       # Energy
            ])
    
    return np.array(features)


def visualize_wavelet_decomposition(image, wavelet='db1', level=2):
    """Hiển thị phân rã wavelet của ảnh"""
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    h, w = image.shape
    new_h = 2 ** int(np.ceil(np.log2(h)))
    new_w = 2 ** int(np.ceil(np.log2(w)))
    image_resized = cv2.resize(image, (new_w, new_h))
    
    coeffs = pywt.wavedec2(image_resized, wavelet=wavelet, level=level)
    arr, slices = pywt.coeffs_to_array(coeffs)
    
    plt.figure(figsize=(10, 8))
    plt.imshow(np.abs(arr), cmap='gray')
    plt.title(f'Wavelet Decomposition (Level {level}, {wavelet})')
    plt.colorbar()
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    return coeffs

## 3. Hàm tính độ tương đồng giữa 2 ảnh

In [ ]:
def calculate_similarity(image1, image2, wavelet='db1', level=3, method='euclidean'):
    """
    Tính độ tương đồng giữa 2 ảnh dựa trên đặc trưng wavelet
    
    Parameters:
        image1, image2: Hai ảnh cần so sánh
        wavelet: Loại wavelet
        level: Cấp độ phân rã
        method: Phương pháp tính khoảng cách ('euclidean', 'cosine', 'correlation')
    
    Returns:
        Khoảng cách (càng nhỏ càng giống nhau)
    """
    # Trích xuất đặc trưng
    features1 = extract_wavelet_features(image1, wavelet, level)
    features2 = extract_wavelet_features(image2, wavelet, level)
    
    # Tính khoảng cách
    if method == 'euclidean':
        distance = euclidean(features1, features2)
    elif method == 'cosine':
        distance = cosine(features1, features2)
    elif method == 'correlation':
        distance = 1 - np.corrcoef(features1, features2)[0, 1]
    else:
        distance = euclidean(features1, features2)
    
    return distance


def similarity_to_probability(distance, threshold=100):
    """
    Chuyển khoảng cách thành xác suất tương đồng
    Sử dụng hàm sigmoid: P = 1 / (1 + exp(distance / threshold))
    """
    probability = 1 / (1 + np.exp(distance / threshold))
    return probability

## 4. Tạo dữ liệu test (các cặp ảnh)

In [ ]:
def create_test_dataset():
    """Tạo dataset test gồm các cặp ảnh với nhãn (1=tương tự, 0=khác nhau)"""
    # Load ảnh mẫu từ scikit-image
    img1 = data.camera()
    img2 = data.astronaut()
    img3 = data.coins()
    img4 = data.moon()
    img5 = data.text()
    
    # Tạo các biến thể của ảnh (để test độ robust)
    img1_noisy = (img1 + np.random.normal(0, 10, img1.shape)).astype(np.uint8)
    img1_rotated = np.rot90(img1)
    img1_scaled = cv2.resize(cv2.resize(img1, (300, 300)), (512, 512))
    img1_bright = np.clip(img1 * 1.3, 0, 255).astype(np.uint8)
    
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
    img2_noisy = (img2_gray + np.random.normal(0, 5, img2_gray.shape)).astype(np.uint8)
    img2_rotated = np.rot90(img2_gray)
    
    # Các cặp TƯƠNG TỰ (Label = 1)
    similar_pairs = [
        (img1, img1_noisy, "Camera - Camera nhiễu"),
        (img1, img1_rotated, "Camera - Camera xoay"),
        (img1, img1_scaled, "Camera - Camera scale"),
        (img1, img1_bright, "Camera - Camera sáng"),
        (img2_gray, img2_noisy, "Astronaut - Astronaut nhiễu"),
        (img2_gray, img2_rotated, "Astronaut - Astronaut xoay"),
    ]
    
    # Các cặp KHÁC NHAU (Label = 0)
    dissimilar_pairs = [
        (img1, img2_gray, "Camera - Astronaut"),
        (img1, img3, "Camera - Coins"),
        (img1, img4, "Camera - Moon"),
        (img1, img5, "Camera - Text"),
        (img2_gray, img3, "Astronaut - Coins"),
        (img2_gray, img4, "Astronaut - Moon"),
        (img3, img4, "Coins - Moon"),
        (img3, img5, "Coins - Text"),
        (img4, img5, "Moon - Text"),
    ]
    
    # Tổng hợp dataset
    image_pairs = []
    labels = []
    
    for pair in similar_pairs:
        image_pairs.append((pair[0], pair[1], pair[2]))
        labels.append(1)
    
    for pair in dissimilar_pairs:
        image_pairs.append((pair[0], pair[1], pair[2]))
        labels.append(0)
    
    return image_pairs, np.array(labels)


# Tạo dataset
image_pairs, true_labels = create_test_dataset()
print(f"Tổng số cặp ảnh: {len(image_pairs)}")
print(f"Số cặp tương tự: {np.sum(true_labels == 1)}")
print(f"Số cặp khác nhau: {np.sum(true_labels == 0)}")

## 5. Hiển thị một số cặp ảnh mẫu

In [ ]:
# Hiển thị một số cặp ảnh
fig, axes = plt.subplots(3, 4, figsize=(16, 12))

for idx in range(6):
    img1, img2, desc = image_pairs[idx]
    label = "TƯƠNG TỰ" if true_labels[idx] == 1 else "KHÔNG TƯƠNG TỰ"
    
    row = idx // 2
    col = (idx % 2) * 2
    
    axes[row, col].imshow(img1, cmap='gray')
    axes[row, col].set_title(f'Ảnh 1', fontsize=10)
    axes[row, col].axis('off')
    
    axes[row, col+1].imshow(img2, cmap='gray')
    axes[row, col+1].set_title(f'Ảnh 2\n{desc}\n[{label}]', fontsize=10, 
                                color='green' if label == "TƯƠNG TỰ" else 'red')
    axes[row, col+1].axis('off')

plt.tight_layout()
plt.savefig('results/sample_pairs.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Tính toán điểm tương đồng cho tất cả các cặp

In [ ]:
# Tính khoảng cách cho tất cả các cặp
distances = []
probabilities = []

print("Đang tính toán độ tương đồng...")
for idx, (img1, img2, desc) in enumerate(image_pairs):
    distance = calculate_similarity(img1, img2, wavelet='db1', level=3, method='euclidean')
    prob = similarity_to_probability(distance, threshold=50)
    
    distances.append(distance)
    probabilities.append(prob)
    
    print(f"Cặp {idx+1}/{len(image_pairs)}: {desc[:30]:30s} | Distance: {distance:8.2f} | Prob: {prob:.4f} | Label: {true_labels[idx]}")

distances = np.array(distances)
probabilities = np.array(probabilities)

print("\n✓ Hoàn thành tính toán độ tương đồng!")

## 7. Tính toán các metrics đánh giá

In [ ]:
def calculate_metrics_at_threshold(distances, true_labels, threshold):
    """
    Tính metrics tại một ngưỡng cụ thể
    distance < threshold => Dự đoán = 1 (Tương tự)
    distance >= threshold => Dự đoán = 0 (Khác nhau)
    """
    predictions = (distances < threshold).astype(int)
    
    # Confusion Matrix
    tn, fp, fn, tp = confusion_matrix(true_labels, predictions).ravel()
    
    # Các metrics
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0  # TPR/Recall
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0  # TNR
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    
    return {
        'threshold': threshold,
        'accuracy': accuracy,
        'sensitivity': sensitivity,
        'specificity': specificity,
        'precision': precision,
        'TP': tp, 'TN': tn, 'FP': fp, 'FN': fn,
        'predictions': predictions
    }


# Tìm ngưỡng tối ưu (maximize accuracy)
thresholds_to_test = np.linspace(distances.min(), distances.max(), 50)
best_metrics = None
best_accuracy = 0

for thresh in thresholds_to_test:
    metrics = calculate_metrics_at_threshold(distances, true_labels, thresh)
    if metrics['accuracy'] > best_accuracy:
        best_accuracy = metrics['accuracy']
        best_metrics = metrics

print("=" * 70)
print("KẾT QUẢ ĐÁNH GIÁ TẠI NGƯỠNG TỐI ƯU")
print("=" * 70)
print(f"\nNgưỡng tối ưu: {best_metrics['threshold']:.2f}")
print(f"\n{'Metric':<20} {'Giá trị':<15} {'Ý nghĩa'}")
print("-" * 70)
print(f"{'Accuracy':<20} {best_metrics['accuracy']:.4f} ({best_metrics['accuracy']*100:.2f}%)     Tỷ lệ phân loại đúng")
print(f"{'Sensitivity':<20} {best_metrics['sensitivity']:.4f} ({best_metrics['sensitivity']*100:.2f}%)     Phát hiện cặp tương tự")
print(f"{'Specificity':<20} {best_metrics['specificity']:.4f} ({best_metrics['specificity']*100:.2f}%)     Phát hiện cặp khác nhau")
print(f"{'Precision':<20} {best_metrics['precision']:.4f} ({best_metrics['precision']*100:.2f}%)     Độ chính xác dự đoán")
print("\nConfusion Matrix:")
print(f"  True Positive (TP):  {best_metrics['TP']:3d} - Tương tự → Tương tự")
print(f"  True Negative (TN):  {best_metrics['TN']:3d} - Khác nhau → Khác nhau")
print(f"  False Positive (FP): {best_metrics['FP']:3d} - Khác nhau → Tương tự (sai)")
print(f"  False Negative (FN): {best_metrics['FN']:3d} - Tương tự → Khác nhau (sai)")
print("=" * 70)

## 8. Vẽ Confusion Matrix

In [ ]:
# Vẽ Confusion Matrix
cm = confusion_matrix(true_labels, best_metrics['predictions'])

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Không tương tự', 'Tương tự'],
            yticklabels=['Không tương tự', 'Tương tự'],
            cbar_kws={'label': 'Số lượng'})

plt.xlabel('Dự đoán', fontsize=12, weight='bold')
plt.ylabel('Thực tế', fontsize=12, weight='bold')
plt.title(f'Confusion Matrix\n(Threshold = {best_metrics["threshold"]:.2f}, Accuracy = {best_metrics["accuracy"]:.2%})', 
          fontsize=14, weight='bold')

# Thêm annotations
for i in range(2):
    for j in range(2):
        plt.text(j + 0.5, i + 0.7, 
                f'{cm[i, j]}', 
                ha='center', va='center', 
                fontsize=20, weight='bold', color='white' if cm[i, j] > cm.max()/2 else 'black')

plt.tight_layout()
plt.savefig('results/confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Vẽ đường cong ROC (Receiver Operating Characteristic)

In [ ]:
# Tính ROC curve (sử dụng probabilities thay vì distances)
fpr, tpr, thresholds_roc = roc_curve(true_labels, probabilities)
roc_auc = auc(fpr, tpr)

# Tìm điểm tối ưu (Youden's Index = TPR - FPR)
youden_index = tpr - fpr
optimal_idx = np.argmax(youden_index)
optimal_threshold_roc = thresholds_roc[optimal_idx]
optimal_fpr = fpr[optimal_idx]
optimal_tpr = tpr[optimal_idx]

# Vẽ ROC curve
plt.figure(figsize=(10, 8))

# Đường ROC
plt.plot(fpr, tpr, color='darkorange', lw=2, 
         label=f'ROC curve (AUC = {roc_auc:.3f})')

# Đường tham chiếu (random classifier)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', 
         label='Random Classifier (AUC = 0.500)')

# Điểm tối ưu
plt.plot(optimal_fpr, optimal_tpr, 'ro', markersize=10, 
         label=f'Optimal Point (TPR={optimal_tpr:.3f}, FPR={optimal_fpr:.3f})')

# Cấu hình
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1 - Specificity)', fontsize=12, weight='bold')
plt.ylabel('True Positive Rate (Sensitivity)', fontsize=12, weight='bold')
plt.title('Receiver Operating Characteristic (ROC) Curve\nWavelet-based Image Similarity', 
          fontsize=14, weight='bold')
plt.legend(loc="lower right", fontsize=11)
plt.grid(alpha=0.3)

# Thông tin bổ sung
textstr = f'AUC = {roc_auc:.4f}\nOptimal Threshold = {optimal_threshold_roc:.4f}\nTPR = {optimal_tpr:.4f}\nFPR = {optimal_fpr:.4f}'
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
plt.text(0.6, 0.2, textstr, fontsize=10, verticalalignment='top', bbox=props)

plt.tight_layout()
plt.savefig('results/roc_curve.png', dpi=150, bbox_inches='tight')
plt.show()

# Giải thích AUC
print(f"\n{'='*70}")
print(f"AUC (Area Under Curve): {roc_auc:.4f}")
print(f"{'='*70}")
print(f"\nĐánh giá AUC:")
print(f"  • 0.9 - 1.0: Xuất sắc")
print(f"  • 0.8 - 0.9: Tốt")
print(f"  • 0.7 - 0.8: Khá")
print(f"  • 0.5 - 0.7: Trung bình")
print(f"  • < 0.5: Kém")

## 10. Phân tích chi tiết hiệu suất ở các ngưỡng khác nhau

In [ ]:
# Tính metrics cho nhiều ngưỡng
test_thresholds = np.linspace(distances.min(), distances.max(), 100)
metrics_list = [calculate_metrics_at_threshold(distances, true_labels, t) for t in test_thresholds]

# Trích xuất các giá trị để vẽ
metrics_data = {
    'Accuracy': ([m['accuracy'] for m in metrics_list], 'b-', best_metrics['accuracy']),
    'Sensitivity': ([m['sensitivity'] for m in metrics_list], 'g-', best_metrics['sensitivity']),
    'Specificity': ([m['specificity'] for m in metrics_list], 'orange', best_metrics['specificity']),
    'Precision': ([m['precision'] for m in metrics_list], 'purple', best_metrics['precision'])
}

# Vẽ biểu đồ sử dụng vòng lặp
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for idx, (metric_name, (values, color, optimal_value)) in enumerate(metrics_data.items()):
    ax = axes[idx]
    
    # Vẽ đường chính
    ax.plot(test_thresholds, values, color, linewidth=2, label=metric_name)
    
    # Đường ngưỡng tối ưu
    ax.axvline(best_metrics['threshold'], color='r', linestyle='--', 
               label=f'Optimal Threshold = {best_metrics["threshold"]:.2f}')
    
    # Đường giá trị tối ưu
    ax.axhline(optimal_value, color='gray', linestyle=':', alpha=0.5)
    
    # Cấu hình
    ax.set_xlabel('Threshold', fontsize=11)
    ax.set_ylabel(metric_name, fontsize=11)
    ax.set_title(f'{metric_name} vs Threshold', fontsize=12, weight='bold')
    ax.grid(alpha=0.3)
    ax.legend()

plt.tight_layout()
plt.savefig('results/metrics_vs_threshold.png', dpi=150, bbox_inches='tight')
plt.show()

## 11. Hiển thị ví dụ về wavelet decomposition

In [ ]:
# Hiển thị wavelet decomposition cho ảnh mẫu
sample_image = data.camera()
print("Wavelet Decomposition:")
visualize_wavelet_decomposition(sample_image, wavelet='db1', level=3)